# [LVV-T2241 (v1.0)] MTAOS corrections accumulation

This notebook is used to analyse the data colleted when running the [LVV-T2216] test case during System Spread Integration Tests on Level 3.  
   
**Requirements:**
 - You should have run this test case and record the time stamps on each step.
 
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

**Make sure you run this notebook on TTS before running at the summit.**

Please, see the [README] file for the requirements to run this notebook.

[LVV-T2241 (v1.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2241
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

In [ ]:
test_case = "LVV-T2241"
test_exec = "LVV-EXXXX"

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

from astropy.time import Time
from matplotlib import pyplot as plt

from lsst.sitcom import vandv

In [ ]:
client = vandv.efd.create_efd_client()

exec_info = vandv.ExecutionInfo()
print(exec_info)

## Check log from the EFD

Use the code below to query the data from the EFD.  
Remember that you can use the index to help selecting the data.  

In [ ]:
date = "2022-07-29"

log_df = await client.select_time_series(
    "lsst.sal.Script.logevent_logMessage", 
    "message",
    Time(f"{date}T00:00:00", format="isot", scale="utc"),
    Time(f"{date}T23:59:00", format="isot", scale="utc"),
    index=-22410729
)

log_df

In [ ]:
t_start = Time(log_df[log_df.message.str.contains("START")].index[0], scale="utc")
t_end = Time(log_df[log_df.message.str.contains("STOP")].index[0], scale="utc")

print(f"Actual test happened between\n {t_start}\n and\n {t_end}\n") 

## Display degrees of freedom

The degrees of freedom are the first step performed by the OFC in converting the wavefront errors into corrections.  

It is composed of two parts, the "aggregated" and the "visit" degrees of freedom.  
The "aggregated" is the combination of all corrections computed so far whereas the "visit" contains only the degrees of freedom from the last correction.    

These values are published as vectors of 50 elements each in the "degreeOfFreedom" event.  
As with the annularZernikeCoeff case above we need to query them individually and then build the vectors afterwards. 

Find more information in the [Document-14771: Active Optics Baseline Design] document, Table 1-1. Telescope degrees of Freedom.

[Document-14771: Active Optics Baseline Design]: ls.st/Document-14771

In [ ]:
aggregated_dof = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_degreeOfFreedom', 
    [f"aggregatedDoF{i}" for i in range(50)], 
    t_start, 
    t_end,
)

In [ ]:
visit_dof = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_degreeOfFreedom', 
    [f"visitDoF{i}" for i in range(50)], 
    t_start, 
    t_end,
)

In [ ]:
fig, axs = fig, axs = plt.subplot_mosaic(
    mosaic="AB\nCC\nDD",
    num="Degrees of Freedom", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 6),
)

fig.suptitle("Degrees of Freedom")
labels = ["1um", "1um+1um", "0um", "2um"]
vandv.mtaos.show_dof(axs["A"], aggregated_dof, "m2hex", labels=labels)
vandv.mtaos.show_dof(axs["B"], aggregated_dof, "camhex", labels=labels)
vandv.mtaos.show_dof(axs["C"], aggregated_dof, "m1m3", labels=labels)
vandv.mtaos.show_dof(axs["D"], aggregated_dof, "m2", labels=labels)

## M1M3 Analysis

Here we want to confirm that the difference between corrections between the 1um+1um position is the same as 2um.

In [ ]:
m1m3_correction = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m1m3Correction', 
    "*",
    t_start, 
    t_end,
)

m1m3_correction = m1m3_correction.select_dtypes(['number'])

In [ ]:
m1m3_correction_1um = m1m3_correction.iloc[0]
m1m3_correction_1um1um = m1m3_correction.iloc[1]
m1m3_correction_2um = m1m3_correction.iloc[3]
m1m3_correction_diff =  m1m3_correction_2um - m1m3_correction_1um1um

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AAA\nBCD\nBCD",
    num="M1M3 Correction Difference", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 6),
)

fig.suptitle("M1M3 Correction Difference")
_ = vandv.m1m3.snapshot_zforces(axs["A"], [m1m3_correction_diff])
_ = vandv.m1m3.snapshot_zforces_overview(axs["B"], m1m3_correction_1um1um, title="1um+1um")
_ = vandv.m1m3.snapshot_zforces_overview(axs["C"], m1m3_correction_2um, title="2um")
_ = vandv.m1m3.snapshot_zforces_overview(axs["D"], m1m3_correction_diff, title="Difference")

In [ ]:
m1m3_applied = await client.select_time_series(
    'lsst.sal.MTM1M3.command_applyActiveOpticForces', 
    [f"zForces{i}" for i in range(156)], 
    t_start, 
    t_end,
)

In [ ]:
m1m3_applied_1um = m1m3_applied.iloc[0]
m1m3_applied_1um1um = m1m3_applied.iloc[1]
m1m3_applied_2um = m1m3_applied.iloc[3]
m1m3_applied_diff =  m1m3_applied_2um - m1m3_applied_1um1um

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AAA\nBCD\nBCD",
    num="M1M3 Corrections Applied Difference", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 6),
)

fig.suptitle("M1M3 Corrections Applied Difference")
_ = vandv.m1m3.snapshot_zforces(axs["A"], [m1m3_applied_diff])
_ = vandv.m1m3.snapshot_zforces_overview(axs["B"], m1m3_applied_1um1um, title="1um+1um")
_ = vandv.m1m3.snapshot_zforces_overview(axs["C"], m1m3_applied_2um, title="2um")
_ = vandv.m1m3.snapshot_zforces_overview(axs["D"], m1m3_applied_diff, title="Difference")

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AAA\nBCD\nBCD",
    num="M1M3 Corrections Vs Applied", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 6),
)

fig.suptitle("M1M3 Corrections Vs Applied")
_ = vandv.m1m3.snapshot_zforces(
    axs["A"], [
        m1m3_correction_1um - m1m3_applied_1um, 
        m1m3_correction_1um1um - m1m3_applied_1um1um, 
        m1m3_correction_2um - m1m3_applied_2um,
    ], 
    labels=["Diff 1um", "Diff 1um+1um", "Diff 2um"]
)

_ = vandv.m1m3.snapshot_zforces_overview(axs["B"], m1m3_correction_1um - m1m3_applied_1um, title="1um+1um")
_ = vandv.m1m3.snapshot_zforces_overview(axs["C"], m1m3_correction_1um1um - m1m3_applied_1um1um, title="2um")
_ = vandv.m1m3.snapshot_zforces_overview(axs["D"], m1m3_correction_2um - m1m3_applied_2um, title="Difference")

## M2 Analysis

Same with m2

In [ ]:
m2_correction = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m2Correction', 
    [f"zForces{i}" for i in range(72)], 
    t_start, 
    t_end,
)

In [ ]:
m2_applied = await client.select_time_series(
    'lsst.sal.MTM2.command_applyForces', 
    [f"axial{i}" for i in range(72)], 
    t_start, 
    t_end,
)

In [ ]:
m2_correction_1um1um = m2_correction.iloc[1]
m2_correction_2um = m2_correction.iloc[3]
m2_correction_diff = m2_correction_2um - m2_correction_1um1um

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="ABC",
    num="M2 Corrections", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 4),
)

fig.suptitle("M2 Corrections")
_ = vandv.m2.snapshot_zforces_overview(axs["A"], m2_correction_1um1um, prefix="zForces", ms=200, fs=6)
_ = vandv.m2.snapshot_zforces_overview(axs["B"], m2_correction_2um, prefix="zForces", ms=200, fs=6)
_ = vandv.m2.snapshot_zforces_overview(axs["C"], m2_correction_diff, prefix="zForces", ms=200, fs=6)

## Analyse CamHex

In [ ]:
el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    "*", 
    t_start,
    t_end)

In [ ]:
# From the XML:
#   Actual MTHexapod position, in order (X, Y, Z, U, V, W). 
#   Linear positions are in microns, angular positions are in degrees.
pos = await client.select_time_series(
    "lsst.sal.MTHexapod.application",
    "*",
    t_start,
    t_end,
    index=1
)

# Unravel in x/y/z/u/v/w
for i, col in enumerate("xyzuvw"):
    pos[col] = pos[f"position{i}"]


# Triggered at the end of a slew
cpos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_compensatedPosition",
    "*",
    t_start,
    t_end,
    index=1
)

# Triggered only after move/offset. Should not see much. 
upos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_uncompensatedPosition",
    "*",
    t_start,
    t_end,
    index=1
)

# Estimate the LUT position
lut_pred = vandv.hexapod.get_lut_positions(index=1, elevation=el.actualPosition)
lut = pd.DataFrame(lut_pred, columns=["x", "y", "z", "u", "v", "w"], index=el.index)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AD\nBE\nCF",
    num="CamHex Analysis", 
    dpi=120,
    figsize=(10, 6),
    sharex=True,
)

cols = "xyzuvw"
for ax, col in zip("ABCDEF", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos, cpos, upos, lut], 
        column=col, 
        elevation=el, 
        symbols=["", "o", "s", ""],
        names=["Actual Position", "Compensated", "Uncompensated", "LUT"]
    )

# Hide xlabel
for i in "ABDE":
    _ = axs[i].set_xlabel("")
    
_ = axs["F"].legend(loc='lower center', bbox_to_anchor=(0.5, -1.0), ncol=5)

fig.suptitle("CamHex Timeline")
fig.autofmt_xdate()
fig.tight_layout()

## Analyse M2Hex

In [ ]:
# From the XML:
#   Actual MTHexapod position, in order (X, Y, Z, U, V, W). 
#   Linear positions are in microns, angular positions are in degrees.
pos = await client.select_time_series(
    "lsst.sal.MTHexapod.application",
    "*",
    t_start,
    t_end,
    index=2
)

# Unravel in x/y/z/u/v/w
for i, col in enumerate("xyzuvw"):
    pos[col] = pos[f"position{i}"]


# Triggered at the end of a slew
cpos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_compensatedPosition",
    "*",
    t_start,
    t_end,
    index=2
)

# Triggered only after move/offset. Should not see much. 
upos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_uncompensatedPosition",
    "*",
    t_start,
    t_end,
    index=2
)

# Estimate the LUT position
lut_pred = vandv.hexapod.get_lut_positions(index=2, elevation=el.actualPosition)
lut = pd.DataFrame(lut_pred, columns=["x", "y", "z", "u", "v", "w"], index=el.index)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AD\nBE\nCF",
    num="M2hex", 
    dpi=120,
    figsize=(10, 6),
    sharex=True,
)

cols = "xyzuvw"
for ax, col in zip("ABCDEF", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos, cpos, upos, lut], 
        column=col, 
        elevation=el, 
        symbols=["", "o", "s", ""],
        names=["Actual Position", "Compensated", "Uncompensated", "LUT"]
    )

# Hide xlabel
for i in "ABDE":
    _ = axs[i].set_xlabel("")
    
_ = axs["F"].legend(loc='lower center', bbox_to_anchor=(0.5, -1), ncol=5)

fig.suptitle("M2Hex Timeline")
fig.autofmt_xdate()
fig.tight_layout()